# Ollama Legal PDF RAG Notebook

## Import Libraries


In [2]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Load PDF
change the path to document for different case files

In [ ]:
# Load PDF

local_path = "sample_data\Ram_Sarup_Gupta_Dead_By_Lrs_vs_Bishun_Narain_Inter_College_Ors_on_8_April_1987.PDF"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

PDF loaded successfully: sample_data\Ram_Sarup_Gupta_Dead_By_Lrs_vs_Bishun_Narain_Inter_College_Ors_on_8_April_1987.PDF


In [4]:
#print(data)

## Split text into chunks

In [5]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Text split into {len(chunks)} chunks")

Text split into 31 chunks


In [6]:
#print(chunks)  # Display first 500 characters of the first chunk

## Create vector database

In [7]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


## Set up LLM and Retrieval

In [8]:
# Set up LLM and retrieval
local_model = "gemma3:4b"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [9]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [10]:
# RAG prompt template
template = """
You are a legal assistant. Use ONLY the information provided in the context below to answer the question as accurately and concisely as possible. Do not make up any information or provide legal advice.

Context:
{context}

Question:
{question}

Answer (based strictly on the context):
"""

prompt = ChatPromptTemplate.from_template(template)

In [11]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [12]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [17]:
chat_with_pdf("What is the main idea of Irrevocable License for School: Ram Sarup Gupta vs Bishun Narain?")

According to the provided text, the main idea of the Irrevocable License for the school (Ram Sarup Gupta vs Bishun Narain) is that the school was able to successfully argue for an irrevocable license under Section 60(b) of the Act, despite not explicitly stating “acting upon the license” in their pleadings. The court upheld this claim because the defendants’ pleadings, when read as a whole, clearly indicated that they had made constructions and incurred expenses *as a result of* the license granted, meeting the legal requirements for an irrevocable license. The court emphasized that the substance of the pleadings was key, rather than precise wording.

In [ ]:
chat_with_pdf("Summarize the arguments regarding the irrevocability of the license.")

Here’s a summary of the arguments regarding the irrevocability of the license, based on the provided text:

**Core Argument for Irrevocability:**

The High Court and ultimately this Court (Supreme Court) held the license was irrevocably granted to the school due to three key factors:

1.  **Permanent Construction:** The school erected works of a permanent nature on the property while operating the school.
2.  **“Acting Upon the License”:** The construction was carried out “acting upon the license,” meaning the school was utilizing the rights granted to them through the license.
3.  **Expenses Incurred:** The school incurred expenses in making the constructions.

**Arguments Against Irrevocability (Rejected):**

*   **Lack of Specific Pleadings:** The appellant (the original litigant) argued that the High Court and trial court didn't frame necessary issues or present sufficient evidence to support their decision. However, the Court rejected this, stating that pleadings didn’t need to reproduce exact wording and that the substance of the respondents’ case was sufficient.
*   **Missing Issues:** The appellant contended that no issue was framed and no evidence was produced by them to support the revocability claim.

**Key Legal Principle:**

The Court emphasized the principle that if a grantor allows another to build on their land in furtherance of a licensed purpose, they cannot later revoke the license, particularly when the conditions of Section 60(b) of the Indian Easements Act (permanent construction, “acting upon the license,” and incurred expenses) are met.

In [ ]:
chat_with_pdf("Can you explain the case study highlighted in the document?")

The document details the case of *Ram Sohanlal Gupta v. Bishun Narain*. Here’s a breakdown of the key elements:

*   **The Dispute:** Bishun Narain had a license to use a property granted by Ram Sohanlal Gupta. Narain then built permanent structures on the property and incurred expenses. The core of the case was whether the license was automatically revoked because Narain had built permanent structures.

*   **Legal Basis:** The case hinges on sections of the Indian Easements Act (specifically sections 60-64) which govern licenses.

*   **Section 60 (Irrevocable Licenses):** The Act outlines conditions under which a license is deemed irrevocable. These include:
    *   The license being coupled with a transfer of property and enforced.
    *   The licensee executing a permanent work and incurring expenses.

*   **The Court’s Ruling:** The court found that *even though* Narain had built permanent structures (meeting one of the conditions for an irrevocable license), the parties could *agree* to make the license revocable despite this.  The court emphasized that the parties’ agreement superseded the automatic implications of the Act. This was based on the principle that if the parties mutually agreed to a different arrangement, it would be upheld.

*   **Key takeaway:** The case highlights that the automatic provisions of the Easements Act regarding irrevocable licenses can be overridden by a specific agreement between the parties involved. The court prioritizes the parties’ expressed wishes over the general rules of the Act.

In [ ]:
chat_with_pdf("tell me the key points in the document")

 1. The document discusses a legal case, Ram Sarup Gupta (Dead) By Lrs vs Bishun Narain Inter College & Ors on 8 April, 1987, regarding a license grant and its revocation.
2. The Indian Easement Act of 1882 is the primary legislation considered in this case.
3. The document discusses various sections of the act, including Sections 52, 60, 62, 63, and 64.
4. It highlights the importance of considering both oral and implied license grants when determining if a license is irrevocable.
5. The conduct of parties and circumstances leading to the granting of a license are crucial in determining its revocability.
6. The document emphasizes the need to focus on the substance of pleadings rather than undue emphasis on their form.

## Chatbot UI

In [18]:
import gradio as gr
import nest_asyncio
nest_asyncio.apply()

def gradio_interface(question, history):
    try:
        response = chain.invoke(question)
    except Exception:
        response = "⚠️ Sorry, something went wrong while processing your request."
    return response

# Gradio UI
demo = gr.ChatInterface(
    fn=gradio_interface,
    title="Legal Document Chatbot",
    description="Ask questions about the Document",
    examples=[
        "tell me the key points in the document.",
        "Summarize the court's reasoning on this case."
    ]
)

# ⚠️ Safely launch Gradio without showing tracebacks
try:
    demo.launch(share=True, show_error=False)  # `show_error=False` hides internal errors
except Exception:
    pass  


* Running on local URL:  http://127.0.0.1:7864


## Clean up (optional)

In [ ]:
# Optional: Clean up when done 
#vector_db.delete_collection()
#print("Vector database deleted successfully")